In [1]:
pip install -U datasets

  Attempting uninstall: datasets
    Found existing installation: datasets 3.5.1
    Uninstalling datasets-3.5.1:
      Successfully uninstalled datasets-3.5.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
import pandas as pd


c:\Users\micha\OneDrive\Desktop\Università\Progetti\IronyDetection\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds = load_dataset("Multilingual-Perspectivist-NLU/MultiPICo")

Generating train split: 100%|██████████| 94342/94342 [00:01<00:00, 79124.82 examples/s]


In [4]:
dataset = ds["train"]
df = dataset.to_pandas()

COLONNE df_ita \

"label" = label assegnata da ciascun annotatore al testo \
"timestamp" = data e ora di completamento dell'annotazione (a livello di testo) \
"level" = livello del commento (raramente ci sono commenti di secondo livello. In quel caso il rapporto tra post e reply rischia di essere meno chiaro) \
"source" = fonte del commento (reddit o twitter) \
"reply" = testo a cui gli annonatori hanno assegnato la label \
"post" = contesto della reply \
"language" = lingua del testo \
"language_variety" = varietà linguistica del testo \
"post_id" = id del post \
"majority_vote" = majority label \
\
\
DATI DEMOGRAFICI: \
'Age' = età di ogni annotatore (il task è stato completato nel 2023) \
'Gender' = genere binario \
'Ethnicity simplified' = etnia \
'Country of birth' = dove è nato \
'Country of residence' = dove risiede \
'Nationality' = nazionalità. Questa colonna è stata utilizzata per inferira la prima lingua dell'annotatore \
'Student status' = label binaria (Yes, No) \
'Employment status' = più opzioni possibili, con un buon grado di granularità \
'Generation' = Generazione di ciascun annotatore \
'GenerationAggregated' = binario (Young, Old) \
'Employed' = Employment status aggregato (Yes, No)



In [5]:
df_ita = df[df["language"] == "it"]
df_ita["label"] = df["label"].astype(int)
df_ita.shape

C:\Users\micha\AppData\Local\Temp\ipykernel_8324\501556012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ita["label"] = df["label"].astype(int)


(4790, 22)

In [6]:
# Majority vote function
def compute_majority(series):
    count_0 = (series == 0).sum()
    count_1 = (series == 1).sum()
    if count_0 > count_1:
        return 0
    elif count_1 > count_0:
        return 1
    else:
        return 'both'

# Compute majority per group and map back
majority_votes = df_ita.groupby('reply_id')['label'].apply(compute_majority)



In [7]:
df_ita['majority_vote'] = df_ita['reply_id'].map(majority_votes)

C:\Users\micha\AppData\Local\Temp\ipykernel_8324\188802293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ita['majority_vote'] = df_ita['reply_id'].map(majority_votes)


-------------------------------------------
\
COLONNE df_ita_gold \

"level" = livello del commento (raramente ci sono commenti di secondo livello. In quel caso il rapporto tra post e reply rischia di essere meno chiaro) \
"source" = fonte del commento (reddit o twitter) \
"reply" = testo a cui gli annonatori hanno assegnato la label \
"post" = contesto della reply \
"language" = lingua del testo \
"language_variety" = varietà linguistica del testo \
"reply_id" = id della reply \
"post_id" = id del post \
"majority_vote" = majority label

In [8]:
df_ita_gold = df_ita.drop_duplicates(subset="reply_id")
df_ita_gold = df_ita_gold[['level', 'source', 'reply', 'post', 'language',
       'language_variety', 'reply_id',
       'post_id', 'majority_vote']]

print(df_ita_gold["majority_vote"].value_counts())

majority_vote
0       722
1       245
both     33
Name: count, dtype: int64


In [9]:
df_ita.to_csv("multipico_ita.csv")
df_ita_gold.to_csv("multipico_ita_gold.csv")

In [10]:
df_ita_gold

,level,source,reply,post,language,language_variety,reply_id,post_id,majority_vote
14171,1.0,twitter,@USER ... O esaltandolo..,Il silenzio e come un fiume di sabbia ...,it,it,9496337119089889126,7357414994945043930,0
14172,1.0,reddit,"Calma, queste sono direttive date proprio dall...","Se il commesso continuerà a fare muro, egli sa...",it,it,15597092344066966299,6535513880047527192,0
14173,2.0,twitter,@USER @USER Anche per rompere il ghiaccio,@USER Gianca già che ci sei regalale un paio d...,it,it,15902869978243358740,7961696982295378594,1
14174,1.0,reddit,>PS se hai la pelle sensibile evita il rasoio ...,Io uso sia il rasoio di sicurezza che lo shave...,it,it,13889809461410309670,9739175578440028850,0
14175,1.0,reddit,Lo dissi tempo fa in un thread analogo: gli HR...,Non l'ultimo che ho fatto (l'ultimo che ho fat...,it,it,4110758050476660806,12650296792333791569,0
...,...,...,...,...,...,...,...,...,...
17124,1.0,twitter,@USER In teoria prude perché la ferita si sta ...,ragaaa è normale che mi prede tutto il braccio...,it,it,7947987642681870192,16233718987500342344,0
17160,2.0,twitter,@USER Buongiorno ❄️☕️??,@USER Buongiorno ⛈️⛈️,it,it,17727788295477306407,9676017290322369006,0
17222,1.0,reddit,"Se non sono parigini, ti troverai bene /s\nCer...",Tra pochi giorni dovrei partire per la Francia...,it,it,10645757515167341739,18264849717665170848,0
17243,1.0,reddit,Lol wut? Che tranquillità che stanno seminando...,Mi manda al manicomio tutta questa tranquillit...,it,it,9796451987259776883,16670462493773561266,both


In [11]:
# Save the dataframe to a CSV file
df_ita_gold.to_csv('df_ita_gold.csv', index=False)

In [12]:
# prompt: save the csv if the rows have majority vote equals to 1 or both. print the number of the roews

# Filter rows where 'majority_vote' is 1 or 'both'
df_filtered = df_ita_gold[df_ita_gold['majority_vote'].isin([1, 'both'])]

# Print the number of rows in the filtered dataframe
print(f"Number of rows with majority vote 1 or both: {len(df_filtered)}")

# Save the filtered dataframe to a new CSV file
df_filtered.to_csv('multipico_ita_filtered.csv', index=False)

Number of rows with majority vote 1 or both: 278
